In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
import numpy as np


In [3]:
transform = transforms.Compose([transforms.ToTensor(),transforms.Normalize([0.5],[0.5])])
mnist = torchvision.datasets.FashionMNIST(root='./datasets',train=True,download=True,transform=transform)
dataloader = DataLoader(mnist,batch_size=128,shuffle=True)

In [4]:
dataloader = DataLoader(mnist,batch_size=128,shuffle=True)

In [5]:


class Generator(nn.Module):
    def __init__(self, input_size=100, hidden_size=128, output_size=28*28):
        super(Generator, self).__init__()
        self.main = nn.Sequential(
            nn.Linear(input_size, hidden_size),
            nn.ReLU(True),
            nn.Linear(hidden_size, hidden_size * 2),
            nn.ReLU(True),
            nn.Linear(hidden_size * 2, hidden_size * 4),
            nn.ReLU(True),
            nn.Linear(hidden_size * 4, output_size),
            nn.Tanh()  # 输出范围 [-1, 1]
        )
    
    def forward(self, x):
        return self.main(x)

class Discriminator(nn.Module):
    def __init__(self, input_size=28*28, hidden_size=32):
        super(Discriminator, self).__init__()
        self.main = nn.Sequential(
            nn.Linear(input_size, hidden_size * 4),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(hidden_size * 4, hidden_size * 2),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(hidden_size * 2, hidden_size),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(hidden_size, 1),
            nn.Sigmoid()
        )
    
    def forward(self, x):
        return self.main(x)

In [6]:
# if __name__ == '__main__':
#     # 定义生成器和判别器
#     G = Generator(input_size=128, hidden_size=128, output_size=28)
#     D = Discriminator(input_size=28*28, hidden_size=32)

#     # 示例输入
#     z = torch.randn(1, 128)  # 噪声输入
#     fake_image = G(z)        # 生成的假图像

#     # 输出生成器的图像形状
#     print("生成的图像形状:", fake_image.shape)  # 应输出 [1, 1, 28, 28]
    
#     # 将图像展平为 [batch_size, 28*28] 以输入判别器
#     fake_image_flat = fake_image.view(-1, 28*28)
#     output = D(fake_image_flat)  # 判别器输出
#     print("判别器输出形状:", output.shape)  # 应输出 [1, 1]

In [ ]:
def imshow(img):
    img = img / 2 + 0.5  # 逆归一化
    npimg = img.numpy()
    plt.imshow(npimg, cmap='gray')  # 使用灰度显示
    plt.show()

if __name__ == '__main__':
    G = Generator(input_size=100, hidden_size=128, output_size=28*28)
    D = Discriminator(input_size=28*28, hidden_size=32)
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(device)
    G.to(device)
    D.to(device)
    
    transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize([0.5], [0.5])])
    mnist = torchvision.datasets.FashionMNIST(root='./datasets', train=True, download=True, transform=transform)
    dataloader = DataLoader(mnist, batch_size=32, shuffle=True)
    
    criterion = nn.BCELoss()
    learning_rate = 0.0002

    # 优化器
    optimizer_g = optim.Adam(G.parameters(), lr=learning_rate)
    optimizer_d = optim.Adam(D.parameters(), lr=learning_rate)
    num_epochs = 50
    
    for epoch in range(num_epochs):
        for i, (images, _) in enumerate(dataloader):
            # 准备真实数据和生成器输入
            images = images.view(images.size(0), -1).to(device)  # 展平图像
            real_labels = torch.ones(images.size(0), 1).to(device)
            fake_labels = torch.zeros(images.size(0), 1).to(device)
            
            # 训练判别器
            outputs = D(images)
            d_loss_real = criterion(outputs, real_labels)
            real_score = outputs
            
            # 生成假图像
            z = torch.randn(images.size(0), 100).to(device)
            fake_images = G(z)
            outputs = D(fake_images.detach())
            d_loss_fake = criterion(outputs, fake_labels)
            fake_score = outputs
            
            # 总判别器损失并反向传播
            d_loss = d_loss_real + d_loss_fake
            optimizer_d.zero_grad()
            d_loss.backward()
            optimizer_d.step()
            
            # 训练生成器
            outputs = D(fake_images)
            g_loss = criterion(outputs, real_labels)
            
            optimizer_g.zero_grad()
            g_loss.backward()
            optimizer_g.step()
            
            # 每100个batch打印一次损失
            
        print(f'Epoch [{epoch}/{num_epochs}], '
                f'd_loss: {d_loss.item():.4f}, g_loss: {g_loss.item():.4f}, '
                f'D(x): {real_score.mean().item():.4f}, D(G(z)): {fake_score.mean().item():.4f}')
        
         # 显示生成的图像
        with torch.no_grad():
            z = torch.randn(1, 100).to(device)  # 使用单个随机噪声生成一张图像
            fake_image = G(z).view(28, 28).cpu()
            imshow(fake_image)

cuda
Epoch [0/50], d_loss: 0.4384, g_loss: 1.8863, D(x): 0.8537, D(G(z)): 0.1798


: 